<a href="https://colab.research.google.com/github/modhudeb/Named-Entity-Recognition-LSTM/blob/main/text_entity_p2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install contractions

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

In [ ]:
import os
import gensim
import contractions
import re

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Loading Data

In [ ]:
os.chdir('drive/MyDrive/data')

In [ ]:
data = pd.read_csv('merged_wiki_ner.csv')

# Code

## Preprocess

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4548181 entries, 0 to 4548180
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   sentences  int64 
 1   words      object
 2   POS        object
 3   tags       object
dtypes: int64(1), object(3)
memory usage: 138.8+ MB


In [ ]:
data['tags'] = data['tags'].apply(lambda x : x.upper())
data['tags'].unique()

array(['I-MISC', 'O', 'I-PER', 'I-ORG', 'I-LOC', 'B-MISC', 'B-PER',
       'B-LOC', 'B-ORG', 'B-GEO', 'B-GPE', 'I-GEO', 'B-TIM', 'B-ART',
       'I-ART', 'I-GPE', 'I-TIM', 'B-NAT', 'B-EVE', 'I-EVE', 'I-NAT'],
      dtype=object)

In [ ]:
# We don't need parts of speech
data.drop(['POS'], axis = 1, inplace = True)

# All sentences index
sentInd = data['sentences'].unique()

 # Label encoding tags column
encoder = LabelEncoder()
data['tags_num'] = encoder.fit_transform(data['tags'])

In [ ]:
tagsLabel = data['tags_num'].unique()
tagsLabel

array([15, 20, 18, 17, 14,  5,  8,  4,  7,  2,  3, 12,  9,  0, 10, 13, 19,
        6,  1, 11, 16])

In [ ]:
tagsName = data['tags'].unique()
tagsName

array(['I-MISC', 'O', 'I-PER', 'I-ORG', 'I-LOC', 'B-MISC', 'B-PER',
       'B-LOC', 'B-ORG', 'B-GEO', 'B-GPE', 'I-GEO', 'B-TIM', 'B-ART',
       'I-ART', 'I-GPE', 'I-TIM', 'B-NAT', 'B-EVE', 'I-EVE', 'I-NAT'],
      dtype=object)

In [ ]:
# checking null
data.isna().sum()

sentences      0
words        957
tags           0
tags_num       0
dtype: int64

In [ ]:
# removing null indices
data.dropna(inplace = True)

In [ ]:
df = data.groupby('sentences')

In [ ]:
df.get_group(2)

,sentences,words,tags,tags_num
56,2,From,O,20
57,2,this,O,20
58,2,climate,O,20
59,2,William,I-PER,18
60,2,Godwin,I-PER,18
61,2,developed,O,20
62,2,what,O,20
63,2,many,O,20
64,2,consider,O,20
65,2,the,O,20


In [ ]:
df.get_group(0)['words'][0]

'The'

In [ ]:
# We want every words of a sentence into an array
# and their corresponding tags
words = []
tags = []
tags_num = []
for x in range(len(data['sentences'].unique())):
  tmp =  []
  for w in df.get_group(x)['words']:
    tmp.append(w.lower())
  if len(tmp) >= 900:
    words.append(tmp[0:300])
    words.append(tmp[300:600])
    # words.append(tmp[600:900])
  elif len(tmp) >= 600:
    words.append(tmp[0:300])
    words.append(tmp[300:600])
    # words.append(tmp[600:len(tmp)])
  elif len(tmp) >= 300:
    words.append(tmp[0:300])
    words.append(tmp[300:len(tmp)])
  else :
    words.append(tmp)

  tmp = []
  for t in df.get_group(x)['tags']:
    tmp.append(t)
  if len(tmp) >= 900:
    tags.append(tmp[0:300])
    tags.append(tmp[300:600])
    # tags.append(tmp[600:900])
  elif len(tmp) >= 600:
    tags.append(tmp[0:300])
    tags.append(tmp[300:600])
    # tags.append(tmp[600:len(tmp)])
  elif len(tmp) >= 300:
    tags.append(tmp[0:300])
    tags.append(tmp[300:len(tmp)])
  else :
    tags.append(tmp)

  tmp = []
  for t in df.get_group(x)['tags_num']:
    tmp.append(t)
  if len(tmp) >= 900:
    tags_num.append(tmp[0:300])
    tags_num.append(tmp[300:600])
    # tags_num.append(tmp[600:900])
  elif len(tmp) >= 600:
    tags_num.append(tmp[0:300])
    tags_num.append(tmp[300:600])
    # tags_num.append(tmp[600:len(tmp)])
  elif len(tmp) >= 300:
    tags_num.append(tmp[0:300])
    tags_num.append(tmp[300:len(tmp)])
  else :
    tags_num.append(tmp)


words = pd.Series(words, name = 'words')
tags = pd.Series(tags, name = 'tags')
tags_num = pd.Series(tags_num, name = 'tags_num')

print('Done')

Done


In [ ]:
data = pd.DataFrame({'words':words, 'tags':tags, 'tags_num':tags_num})
del(words, tags, tags_num)

In [ ]:
data.to_csv('final_ner_data2.csv', index = False)

In [ ]:
df = data.copy()
df.head(2)

,words,tags,tags_num
0,"[the, oxford, companion, to, philosophy, says,...","[I-MISC, I-MISC, I-MISC, I-MISC, I-MISC, O, O,...","[15, 15, 15, 15, 15, 20, 20, 20, 20, 20, 20, 2..."
1,"[in, the, end, ,, for, anarchist, historian, d...","[O, O, O, O, O, O, O, I-PER, I-PER, O, O, O, O...","[20, 20, 20, 20, 20, 20, 20, 18, 18, 20, 20, 2..."


In [ ]:
" ".join(df['words'][0])

'the oxford companion to philosophy says , " there is no single defining position that all anarchists hold , and those considered anarchists at best share a certain family resemblance . "'

# Word2Vec embedding

In [ ]:
wvmodel = gensim.models.Word2Vec(window=4, min_count=1, vector_size = 64)
wvmodel.build_vocab(df['words'])
# Un-comment the following line if you want word2vec embedding for model and use "first way" in the later step
# model.train(words, epochs = 10, total_examples=model.corpus_count)

In [ ]:
# model.wv.most_similar(['facebook'])

In [ ]:
wvmodel.save('w2vmodel_ner.bin')

# MODEL

In [ ]:
df = pd.read_csv('final_ner_data.csv')
df['words'] = df['words'].apply(lambda x : eval(x))
df['tags_num'] = df['tags_num'].apply(lambda x : eval(x))

In [ ]:
wvmodel = gensim.models.Word2Vec.load('w2vmodel_ner.bin')

## : Pre stage

In [ ]:
df['words'].head(2)

0    [the, oxford, companion, to, philosophy, says,...
1    [in, the, end, ,, for, anarchist, historian, d...
Name: words, dtype: object

<h4> Here we have 2 ways : Either assign corresponding vector or correspond index of the word

In [ ]:
# # First way : assigning vectors of the words
# def vectorizer(X):
#   X = [model.wv[word] for word in X]
#   return X

# words_vectored = words.apply(vectorizer)


In [ ]:
# Second way : assigning index of the words
indexof = wvmodel.wv.key_to_index
def vectorizer(X):
  X = [indexof[word] for word in X]
  return X

words_vectored = df['words'].apply(vectorizer)


In [ ]:
from keras.utils import to_categorical

In [ ]:
# We need to use padded sequence
max_len = max([len(X) for X in df['words']])
print(max_len)
#
words_vectored_padded = pad_sequences(words_vectored[0:120000], maxlen= max_len, padding = 'post', truncating = 'post', value = indexof['.'])
labels_padded = pad_sequences(df['tags_num'][0:120000], maxlen= max_len, padding = 'post', truncating = 'post', value = 20)
# labels_padded = np.array([to_categorical(i, num_classes=len(tagsLabel)) for i in labels_padded])
labels_padded = np.array([to_categorical(i, num_classes=21) for i in labels_padded])

300


In [ ]:
# train test split
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(words_vectored_padded, labels_padded, test_size=0.3, random_state=42)


In [ ]:
# del(words_vectored, data, words_vectored_padded, labels_padded)
del(words_vectored, words_vectored_padded, labels_padded)

In [ ]:
print(xtrain.shape)
print(ytrain.shape)

(84000, 300)
(84000, 300, 21)


## : Final stage

### Fine from here

In [ ]:
tagsName = ['I-MISC', 'O', 'I-PER', 'I-ORG', 'I-LOC', 'B-MISC', 'B-PER',
       'B-LOC', 'B-ORG', 'B-GEO', 'B-GPE', 'I-GEO', 'B-TIM', 'B-ART',
       'I-ART', 'I-GPE', 'I-TIM', 'B-NAT', 'B-EVE', 'I-EVE', 'I-NAT']
tagsLabel = [15, 20, 18, 17, 14,  5,  8,  4,  7,  2,  3, 12,  9,  0, 10, 13, 19,
        6,  1, 11, 16]

In [ ]:
print(tagsName)
print(tagsLabel)
print("Total : ",len(tagsLabel))

['I-MISC', 'O', 'I-PER', 'I-ORG', 'I-LOC', 'B-MISC', 'B-PER', 'B-LOC', 'B-ORG', 'B-GEO', 'B-GPE', 'I-GEO', 'B-TIM', 'B-ART', 'I-ART', 'I-GPE', 'I-TIM', 'B-NAT', 'B-EVE', 'I-EVE', 'I-NAT']
[15, 20, 18, 17, 14, 5, 8, 4, 7, 2, 3, 12, 9, 0, 10, 13, 19, 6, 1, 11, 16]
Total :  21


In [ ]:
xtrain[0].shape

(300,)

In [ ]:
ytrain[0].shape

(300, 21)

In [ ]:
len(wvmodel.wv.index_to_key)

117936

In [ ]:
# callback function
early_stopping = EarlyStopping(monitor='loss', patience=3)


# model
ner_model = tf.keras.Sequential([
    keras.layers.Embedding(len(wvmodel.wv.index_to_key), 64, input_length = max_len),
    keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
    keras.layers.Dropout(0.3),
    keras.layers.TimeDistributed(keras.layers.Dense(len(tagsLabel), activation='softmax')),
    # keras.layers.TimeDistributed(keras.layers.Dense(21, activation='softmax')),
])

ner_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
ner_model.fit(xtrain, ytrain, batch_size = 32, epochs = 4, callbacks = [early_stopping])

Epoch 1/4
2625/2625 [==============================] - 120s 42ms/step - loss: 0.0367 - accuracy: 0.9916
Epoch 2/4
2625/2625 [==============================] - 68s 26ms/step - loss: 0.0121 - accuracy: 0.9963
Epoch 3/4
2625/2625 [==============================] - 68s 26ms/step - loss: 0.0086 - accuracy: 0.9973
Epoch 4/4
2625/2625 [==============================] - 65s 25ms/step - loss: 0.0066 - accuracy: 0.9979


In [ ]:
# ypred = np.round(ner_model.predict(xtest))

In [ ]:
# ner_model.summary()

In [ ]:
ner_model.evaluate(xtest, ytest)

1125/1125 [==============================] - 12s 10ms/step - loss: 0.0129 - accuracy: 0.9965


[0.012883535586297512, 0.9964663982391357]

### test

In [ ]:
tagsDict = {}
for x,y in zip(tagsLabel, tagsName):
  tagsDict[x] = y

👇 This are some important functions for this dataset - To test a single string

In [ ]:
# Few more necessary Preprocessing
stop_words = ['a', 'an', 'the', 'is', 'am', 'are', 'was', 'were', 'might', 'must','and', 'or', 'either', 'neither']

# Important preprocessing functions for this dataset

def preprocessing_data(X):
  X = contractions.fix(X.lower())
  X = re.sub(r'[^\w\s.]', '', X)
  X = re.sub(r'\s+', ' ', X)
  X = word_tokenize(X)
  # lemm = WordNetLemmatizer()
  # X = [lemm.lemmatize(word) for word in X if word not in stop_words]
  # X = [word for word in X if word not in stop_words]
  return X


def predict(X:str):
  X = preprocessing_data(X)
  txt = X
  worddict = wvmodel.wv.key_to_index
  maxlen = ner_model.get_build_config()['input_shape'][1]
  X = np.array([worddict[x] if x in worddict.keys() else worddict['.'] for x in X])
  X = pad_sequences([X], maxlen=maxlen, padding = 'post')

  # prediction
  X = ner_model.predict(X, verbose=0)
  X = np.round(X)
  tags_num = []
  tags_name = []
  for i in X[0][0:len(txt)]:
    j_index = 0
    for j in i:
      if j == 1 :
        tags_num.append(j_index)
        tags_name.append(tagsDict[j_index])
      j_index+=1

  return (np.array(tags_name), np.array(tags_num))




def getEntity(String:str):
  tags,tagsnum = predict(String)
  String = preprocessing_data(String)
  print(String)
  print("-- Entities -- ")
  cout = 1
  tmp = ' '
  init = 0
  for i in range(len(tags)):
    if tags[i] != "O":
      if tmp != tags[i].split('-')[1]:
        if cout>1:
          print((" ".join(String[init : init+cout])).capitalize(), " : ", tmp)
        tmp = tags[i].split('-')[1]
        init = i
        cout = 1
      else :
        cout+=1
    else:
      if cout>1:
          print((" ".join(String[init : init+cout])).capitalize(), " : ", tmp)
          cout = 1
          tmp = ' '


In [ ]:
getEntity("Albert Einstein was born on March 14, 1879, in Ulm, Germany. He was a renowned physicist known for his theory of relativity. Einstein spent a significant part of his career at the Institute for Advanced Study in Princeton, New Jersey. He was awarded the Nobel Prize in Physics in 1921.")

['albert', 'einstein', 'was', 'born', 'on', 'march', '14', '1879', 'in', 'ulm', 'germany', '.', 'he', 'was', 'a', 'renowned', 'physicist', 'known', 'for', 'his', 'theory', 'of', 'relativity', '.', 'einstein', 'spent', 'a', 'significant', 'part', 'of', 'his', 'career', 'at', 'the', 'institute', 'for', 'advanced', 'study', 'in', 'princeton', 'new', 'jersey', '.', 'he', 'was', 'awarded', 'the', 'nobel', 'prize', 'in', 'physics', 'in', '1921', '.']
-- Entities -- 
Albert einstein  :  PER
Ulm germany  :  LOC
Institute for advanced study  :  LOC
Princeton new jersey  :  LOC
Nobel prize in physics  :  MISC


### MOdel saving

In [ ]:
ner_model.save('NER_MODEL.h5')